In [1]:
import json

with open('../final_data/Trending/train.json','r') as f:
    train_datas = json.load(f)

example_input_list = ['说法：' + data['claim'] + '\n' +'证据：' + data['evidence']['text'] for data in train_datas]

example_output_list = [data['human_revision'] for data in train_datas]

system =  {"role": "system",
"content": f'''
你是一个事实核查专家，擅长判断新闻或某个说法的真实性，并且生成相应的解释。
已知事实核查任务是根据给定的证据来判断当前说法的真实性以及生成相应的解释，包含三种类型（正确/错误/证据不足）。
现在我提供给你当前说法以及证据，请你给出相应的解释。
接下来，我将提供几个例子，你可以参照例子进行输出
示例1输入：{example_input_list[0]}
示例1输出：{example_output_list[0]}
示例2输入：{example_input_list[1]}
示例2输出：{example_output_list[1]}
示例3输入：{example_input_list[2]}
示例3输出：{example_output_list[2]}
示例4输入：{example_input_list[3]}
示例4输出：{example_output_list[3]}
示例5输入：{example_input_list[4]}
示例5输出：{example_output_list[4]}
###
请你根据当前说法和给定证据生成一个解释。
注意：不要使用第一人称!
注意：内容精简，不包含冗余信息!
注意：严格根据证据进行真实性判断，并在解释的最后进行输出，例如“因此，该说法是错误的”
注意：严格基于证据进行解释生成!
'''} 

In [2]:
import requests 
import json
def gen_result(system,message,temperature):
    url = ""
   
    
    messages = [
            system,
            {"role": "user", "content": message},
    ]
    api_key = ""
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    
    model = 'gpt-4o-2024-05-13'#"gpt-4-0125-preview"#'GPT-3.5-Turbo-1106'#"gpt-4-0125-preview"#'GPT-3.5-Turbo-1106'#"gpt-4-0125-preview"#'gpt-4-0125-preview'#"GPT-3.5-Turbo-1106"#"gpt-4-1106-preview"
    
    data = {
        "model": model,
        "messages": messages,
        "temperature":temperature,
        # "tenant":"intelligentInteraction",
        # "velenOfflineChatFlag":"false"
    }
    content = json.dumps(data)
    
    try:
        response = requests.post(url, headers=headers, data=content)
        reply = response.json()['choices'][0]['message']['content']
        return 1,reply
    except Exception as e:
        print(e)
        return 0,""

def request_result(system,message,temperature):
    max_retry_nums = 3
    cnt = 0
    res = ""
    while(cnt<=max_retry_nums):
        flag,res = gen_result(system,message,temperature)
        if flag==1:
            try:
                # load_res = json.loads(res)
                break
            except:
                cnt+=1
        else:
            cnt+=1
            
    return res

In [3]:
with open('../final_data/Trending/test.json','r') as f:
    test_data = json.load(f)

In [5]:
def get_user_message(data):
    user_message = f'当前说法为：{data["claim"]}\n'
    user_message += f'当前证据为：{data["evidence"]["text"]}\n'
    return user_message

In [ ]:
test_dict = {}
for idx,data in enumerate(test_data):
    res = request_result(system,get_user_message(data),0.0)
    print(idx)
    test_dict[idx] = res
with open(f'../test_result/GPT4o_results_temperature0_trending.json','w') as f:
    f.write(json.dumps(test_dict, ensure_ascii=False))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
'choices'
'choices'
'choices'
'choices'
21
22
23
24
25
26
27
28
29
'choices'
'choices'
'choices'
'choices'
30
31
32
33
34
35
36
37
38
39
40
41
42


In [10]:
import anthropic
test_dict = {}

client = anthropic.Anthropic(
    auth_token="",
    base_url=""
)

for idx,data in enumerate(test_data):
    try:
        chat_completion = client.messages.create(
            model="anthropic.claude-3-opus",
            max_tokens=16384,
            system=system['content'], # <-- system prompt
            messages=[
                {"role": "user", "content": get_user_message(data)}
            ],
            temperature = 0.0
        )
        res = chat_completion.content[0].text
    except:
        res = ""
    print(idx)
    test_dict[idx] = res
with open(f'../test_result/claude-3-opus_results_temperature0_trending.json','w') as f:
    f.write(json.dumps(test_dict, ensure_ascii=False))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
import anthropic #仅测试0.18.1版本，前期版本中client.messages可能需要替换为client.beta.messages

#段式调用
client = anthropic.Anthropic(
    auth_token="",
    base_url=""
)

chat_completion = client.messages.create(
    model="anthropic.claude-3-sonnet",
    max_tokens=16384,
    system=system['content'], # <-- system prompt
    messages=[
        {"role": "user", "content": get_user_message(data)}
    ],
    temperature = 0.0
)
print(chat_completion.content[0].text)

根据给定的证据,无法确定早晨是最佳健身时间。

证据1指出,虽然早晨锻炼可以燃烧更多脂肪,但对于增肌和力量训练,傍晚锻炼的效果更好。研究发现,傍晚锻炼组的肌肉增长和去脂体重增加更多。

证据2则表示,最佳锻炼时间因人而异,没有一个放之四海而皆准的最佳时间。关键是要坚持锻炼,而不是过于纠结锻炼时间。

因此,根据现有证据,说法"最好的健身时间是早晨"是错误的。最佳健身时间需要根据个人情况和锻炼目标来确定。


In [ ]:
print(1)

1
